In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from sklearn.datasets import load_iris

iris_data = load_iris()
df = pd.DataFrame(np.c_[ iris_data['data'], iris_data['target'] ],
                    columns=iris_data['feature_names'] + ['target'])

In [3]:
def split_dataset(X, y, test_size, shuffle=True):
    n_samples = X.shape[0]
    permut = np.random.permutation(n_samples) if shuffle else range(n_samples)
    split_point = round((1-test_size)*n_samples)

    X_train, y_train = X[permut[:split_point]], y[permut[:split_point]]
    X_test, y_test = X[permut[split_point:]], y[permut[split_point:]]

    return X_train, X_test, y_train, y_test

In [4]:
def one_hot_encoder(target):
    keys = list(set(target))
    values = list(range(len(keys)))
    class_dict = {key:val for (key, val) in zip(keys, values)}

    target_one_hot = np.zeros((target.shape[0], len(keys)), dtype=int)
    for i, item in enumerate(target):
        target_one_hot[i, class_dict[item]] = 1

    return target_one_hot

In [5]:
X_iris = df.drop(columns=['target']).to_numpy()
y_iris = one_hot_encoder(df['target'].to_numpy())
X_train, X_test, y_train, y_test = split_dataset(X_iris, y_iris, test_size=0.2)

In [6]:
def relu(X):
    return np.maximum(X, 0.)

In [7]:
def softmax(y_hat):
    y_hat = np.exp(y_hat)
    y_hat = y_hat / np.sum(y_hat, axis=0)

    return y_hat

In [9]:
def forward_pass(X, params):
    w_1, b_1, w_2, b_2 = params
    b_1 = b_1.reshape(-1, 1)
    b_2 = b_2.reshape(-1, 1)

    z_1 = w_1 @ X.T + b_1
    a_1 = relu(z_1)
    z_2 = w_2 @ a_1 + b_2
    y_hat = softmax(z_2)

    return y_hat